- 6)**Agentes** Use la clase agente para modelar una epidemia Zombie como se describe en [Zombie Infection Simulator](http://asymptote.wordpress.com/2008/01/13/asymptotes-zombie-infection-simulator/) (Esta incluye humanos que pelean  y humanos en pánico)

In [2]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc, cm
from IPython.display import HTML
import math

random.seed(10)

class Agente:
    """ Agente general """
    def __init__(self, tipo, locacion=np.array([0,0])):
        self.tipo = tipo
        self.locacion = locacion
        self.siguiente_accion = None

    def distancia(self, otro):
        "Calcula la distancia euclídea entre este agente y otro."
        return np.linalg.norm(self.locacion - otro.locacion)  # Otra manera es usar np.sqrt(np.sum((a-b)**2))
    
    def vecinos(self, agentes):
        pass
    
    def actuar(self, agentes):
        self.locacion =  self.siguiente_accion
        
    def decidir(self, agentes):
        self.siguiente_accion = np.array([random.uniform(0, 1), random.uniform(0, 1)]) # Toma una nueva posición al azar
    
    def actualizar(self, agentes):
        self.decidir(agentes)
        self.actuar(agentes)
        
class Mundo:
    def __init__(self, agentes, ancho=8, alto=8, steps = 10):
        self.agentes = agentes
        self.ancho = ancho
        self.alto = alto
        self.steps = steps
        self.init_anim()
        
        print ("Creado el mundo")
        
    def init_anim(self):
        self.fig = plt.figure(figsize=(self.ancho, self.alto))
        self.ax = plt.axes(xlim=(0, 1), ylim=(0, 1))
        plot_args = {'markersize' : 8, 'alpha' : 0.6}
        self.puntos, = self.ax.plot([], [], 'o', **plot_args)
       
       
    def dibujar(self, step):
        x_values_0, y_values_0 = [], []
        for agente in self.agentes:
            x, y = agente.locacion
            x_values_0.append(x)
            y_values_0.append(y)

        self.puntos.set_data(x_values_0, y_values_0)
        
        self.ax.set_title('Paso {}'.format(step))
        
        return self.puntos,
       
    def actualizar(self, step):   
        self.dibujar(step) # Dibuja el mundo
        
        for agente in self.agentes:
            agente.actualizar(self.agentes)
    
    def clean_screen(self):
        self.puntos.set_data([], [])
        return self.puntos,
          
    def simular(self):
        anim = animation.FuncAnimation(self.fig, self.actualizar, init_func=self.clean_screen, frames=self.steps, interval=1000, blit=False)
        return anim
#0=humanos,1=humanos_panico,2=zombies,3=humanos_lucha,4=muerto
class Agente_Mundo_Zombie2(Agente):
    def __init__(self, tipo):
        Agente.__init__(self,tipo)
        self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        self.panico = 0
        self.furia = 0
        
    def actuar(self,agentes):
        v = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        v /= np.linalg.norm(v)
        bandera = 0
        it = 0
        vecino_humano = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        vecino_luchador = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        while bandera == 0 and it < (len(agentes)-1):
            if agentes[it].tipo == 0 and bandera == 0:
                vecino_humano = agentes[it].locacion
                bandera = 1
            elif agentes[it].tipo == 2 and bandera == 0:
                vecino_luchador = agentes[it].locacion 
                bandera = 1
            it += 1
        if self.tipo == 0:
            self.locacion = vecino_humano + v
            
            if self.locacion[0] >= 0.99 or self.locacion[0] <= -0.99 or self.locacion[1] >= 0.99 or self.locacion[1] <= -0.99:
                self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        elif self.tipo == 1:
            self.locacion = self.locacion + v
            if self.locacion[0] >= 0.99 or self.locacion[0] <= -0.99 or self.locacion[1] >= 0.99 or self.locacion[1] <= -0.99:
                self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        elif self.tipo == 2:
            self.locacion = self.locacion + v*1.1
            if self.locacion[0] >= 0.99 or self.locacion[0] <= -0.99 or self.locacion[1] >= 0.99 or self.locacion[1] <= -0.99:
                self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        elif self.tipo == 3:
            self.locacion = vecino_luchador + v
            if self.locacion[0] >= 0.99 or self.locacion[0] <= -0.99 or self.locacion[1] >= 0.99 or self.locacion[1] <= -0.99:
                self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
        else:
            self.locacion = self.locacion
            if self.locacion[0] >= 0.99 or self.locacion[0] <= -0.99 or self.locacion[1] >= 0.99 or self.locacion[1] <= -0.99:
                self.locacion = np.array([random.uniform(-1,1),random.uniform(-1,1)])
    def decidir(self,agentes):
        bandera = 0
        it = 0
        if self.tipo == 0:
            while bandera == 0 and it < (len(agentes)-1):
                distPan = self.distancia(agentes[it])
                if (agentes[it].tipo == 1 or agentes[it].tipo == 2) and distPan < 1:
                    self.panico += 1
                    bandera = 1
                elif agentes[it].tipo == 3 and distPan < 1:
                    self.furia += 1
                    bandera = 1
                it += 1
            if self.furia >= 4:
                self.tipo = 3
            if self.panico >= 4:
                self.tipo = 1
        elif self.tipo == 1:
            for agente in agentes:
                aux = 0 
                distCalma = self.distancia(agente)
                if (agentes[it].tipo == 1 or agentes[it].tipo == 2) and distCalma < 1:
                    aux = 1
                    self.panico += 1
                elif agentes[it].tipo == 3 and disCalma < 1:
                    self.panico -= 1
                if aux == 0:
                    self.panico -= 1
                if self.panico < 4:
                    self.tipo = 0
        elif self.tipo == 2:
            for agente in agentes:
                distAtac = self.distancia(agente)
                if distAtac < 0.05:
                    agente.tipo = 2
        elif self.tipo == 3:
            for agente in agentes:
                distAtac = self.distancia(agente)
                if distAtac < 0.05 and agente.tipo == 2:
                    agente.tipo = 4
            if self.furia < 4:
                self.tipo = 0
        else:
            self.tipo = self.tipo 
            
            
class Mundo_Zombie2(Mundo):
    
    def init_anim(self):
        self.fig = plt.figure(figsize=(self.ancho, self.alto))
        self.ax = plt.axes(xlim=(-1,1), ylim=(-1,1))
        plot_args = {'markersize' : 8, 'alpha' : 0.6}
        self.puntos_0, = self.ax.plot([],[],'o',markerfacecolor="blue")
        self.puntos_1, = self.ax.plot([],[],'o',markerfacecolor="red")
        self.puntos_2, = self.ax.plot([],[],'o',markerfacecolor="green")
        self.puntos_3, = self.ax.plot([],[],'o',markerfacecolor="yellow")
        self.puntos_4, = self.ax.plot([],[],'+',markerfacecolor="black")
        
    def clean_screen(self):
        self.puntos_0.set_data([],[])
        self.puntos_1.set_data([],[])
        self.puntos_2.set_data([],[])
        self.puntos_3.set_data([],[])
        self.puntos_4.set_data([],[])
    
    def dibujar(self,step):
        x_0,y_0=[],[]
        x_1,y_1=[],[]
        x_2,y_2=[],[]
        x_3,y_3=[],[]
        x_4,y_4=[],[]
        
        for agente in self.agentes:
            x,y= agente.locacion
            if agente.tipo == 0:
                x_0.append(x)
                y_0.append(y)
            elif agente.tipo == 1:
                x_1.append(x)
                y_1.append(y)
            elif agente.tipo == 2:
                x_2.append(x)
                y_2.append(y)
            elif agente.tipo == 3:
                x_3.append(x)
                y_3.append(y)
            else:
                x_4.append(x)
                y_4.append(y)
        
        self.puntos_0.set_data(x_0,y_0)
        self.puntos_1.set_data(x_1,y_1)
        self.puntos_2.set_data(x_2,y_2)
        self.puntos_3.set_data(x_3,y_3)
        self.puntos_4.set_data(x_4,y_4)
        
        self.ax.set_title('Paso {}'.format(step))
        
        return 

In [ ]:
num_tipo0 = 99
num_tipo2 = 1

agentes2 = [Agente_Mundo_Zombie2(tipo=0) for i in range(num_tipo0)]
agentes2.append(Agente_Mundo_Zombie2(tipo=2))

mundo_zombie2 = Mundo_Zombie2(agentes = agentes2, steps=100)

sim_zombie2 = mundo_zombie2.simular()

In [ ]:
HTML(sim_zombie2.to_html5_video())